## Importazione librerie

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import shap
import lime
from lime.lime_tabular import LimeTabularExplainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_squared_error
shap.initjs()
import json
import re
import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords
from collections import Counter
from PyPDF2 import PdfReader
import re 
import csv
from rapidfuzz import process, fuzz
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json


## File completo con tutte le domande

In [26]:
dir=os.getcwd()
print(dir)

c:\Users\Maria Beatrice\Desktop\test\UNIVERSITA\SECONDO anno magistrale\BIGDATA\Big-Data-Analytics\Progetto Big Data


In [27]:
#df_preclinica = pd.read_csv(file_path("area_preclinica_2016_2019_finale.csv"))

# Usa il percorso assoluto per il file clinica
PRECLINICA_PATH= r"output\preclinica\area_preclinica_2016_2019_finale.csv"
CLINICA_PATH = r"output\clinica\area_clinica_2016_2019_finale.csv"
df_preclinica= pd.read_csv(PRECLINICA_PATH)
df_clinica = pd.read_csv(CLINICA_PATH)

df_finale = pd.concat([df_preclinica, df_clinica])
df_finale.to_csv("area_preclinica_clinica_2016_2019_finale.csv", index=False, encoding='utf-8')

# passaggio a jsonl
output_file = "area_preclinica_clinica_2016_2019_finale.jsonl"

# Apri file in scrittura
with open(output_file, "w", encoding="utf-8") as f:
    for _, row in df_finale .iterrows():
        # Costruisci dizionario nel formato richiesto
        json_obj = {
            "question": row["question"],
            "options": [
                {"A": row["A"]},
                {"B": row["B"]},
                {"C": row["C"]},
                {"D": row["D"]},
                {"E": row["E"]}
            ],
            "answer": row["answer"],
            "category": row["macro category"] ,
            "macro_category": row["category"]
        }
        # Scrivi ogni riga come JSON
        f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")


## estrazione sottocampione

In [28]:
sample = df_finale.sample(n=50, random_state=42)
sample.to_csv("sample.csv", index=False, encoding='utf-8')

# passaggio a jsonl
output_file = "sample.jsonl"

# Apri file in scrittura
with open(output_file, "w", encoding="utf-8") as f:
    for _, row in sample .iterrows():
        # Costruisci dizionario nel formato richiesto
        json_obj = {
            "question": row["question"],
            "options": [
                {"A": row["A"]},
                {"B": row["B"]},
                {"C": row["C"]},
                {"D": row["D"]},
                {"E": row["E"]}
            ],
            "answer": row["answer"],
            "category": row["macro category"] ,
            "macro_category": row["category"]
        }
        # Scrivi ogni riga come JSON
        f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")


## Apriamo il csv e creiamo una colonna index

In [29]:
df= pd.read_csv("area_preclinica_clinica_2016_2019_finale.csv")

df["index"] = range(len(df))

df=df[["index"] + [col for col in df.columns if col != "index"]]

df = df.drop(["id", "A", "B", "C", "D", "E"], axis=1)

# Mostra il risultato
df.head()

,index,question,answer,category,macro category
0,0,in una polmonite da staphylococcus aureus meti...,A,Area Preclinica 2016,farmacologia
1,1,i farmaci ipoglicemizzanti orali della classe ...,B,Area Preclinica 2016,farmacologia
2,2,"quale tra i seguenti farmaci antidiabetici, de...",D,Area Preclinica 2016,farmacologia
3,3,nello shock emorragico il trattamento iniziale...,A,Area Preclinica 2016,farmacologia
4,4,indicare quale dei seguenti farmaci antitumora...,A,Area Preclinica 2016,farmacologia


## Unire al dataset di interesse le risposte dei modelli


In [30]:
#Gemma
with open("custom_openai_google_gemma_3_27b_it_area_preclinica_clinica_2016_2019_finale_fast.json", "r", encoding="utf-8") as f:
    json_data_gemma = json.load(f)

# Crea un dizionario index → output
output_by_index_gemma = {entry["index"]: entry["output"] for entry in json_data_gemma["results"]}
print(output_by_index_gemma)

#Llama
with open("custom_openai_meta_llama_llama_4_maverick_area_preclinica_clinica_2016_2019_finale_fast.json", "r", encoding="utf-8") as f:
    json_data_llama = json.load(f)

# Crea un dizionario index → output
output_by_index_llama = {entry["index"]: entry["output"] for entry in json_data_llama["results"]}
print(output_by_index_llama)

## Mistral

with open("custom_openai_mistralai_mistral_small_3.2_24b_instruct_area_preclinica_clinica_2016_2019_finale_fast.json", "r", encoding="utf-8") as f:
    json_data_mistral = json.load(f)

# Crea un dizionario index → output
output_by_index_mistral = {entry["index"]: entry["output"] for entry in json_data_mistral["results"]}
print(output_by_index_mistral)


{0: 'A', 1: 'B', 2: 'D', 3: 'A', 4: 'A', 5: 'A', 6: 'E', 7: 'B', 8: 'E', 9: 'D', 10: 'A', 11: 'D', 12: 'C', 13: 'C', 14: 'D', 15: 'D', 16: 'C', 17: 'A', 18: 'D', 19: 'E', 20: 'E', 21: 'D', 22: 'A', 23: 'B', 24: 'D', 25: 'D', 26: 'B', 27: 'E', 28: 'B', 29: 'B', 30: 'C', 31: 'D', 32: 'A', 33: 'B', 34: 'C', 35: 'E', 36: 'D', 37: 'C', 38: 'C', 39: 'B', 40: 'A', 41: 'C', 42: 'E', 43: 'E', 44: 'A', 45: 'B', 46: 'B', 47: 'B', 48: 'A', 49: 'C', 50: 'E', 51: 'D', 52: 'D', 53: 'D', 54: 'B', 55: 'D', 56: 'D', 57: 'A', 58: 'B', 59: 'B', 60: 'C', 61: 'D', 62: 'B', 63: 'E', 64: 'B', 65: 'B', 66: 'E', 67: 'B', 68: 'B', 69: 'C', 70: 'D', 71: 'E', 72: 'A', 73: 'B', 74: 'B', 75: 'D', 76: 'C', 77: 'E', 78: 'A', 79: 'D', 80: 'A', 81: 'D', 82: 'E', 83: 'C', 84: 'D', 85: 'E', 86: 'B', 87: 'B', 88: 'D', 89: 'B', 90: 'E', 91: 'D', 92: 'D', 93: 'C', 94: 'B', 95: 'A', 96: 'C', 97: 'D', 98: 'D', 99: 'A', 100: 'C', 101: 'A', 102: 'B', 103: 'A', 104: 'C', 105: 'B', 106: 'C', 107: 'B', 108: 'D', 109: 'D', 110: 'A',

In [37]:
df["gemma"] = df["index"].apply(lambda idx: output_by_index_gemma.get(idx, ""))
df["llama"] = df["index"].apply(lambda idx: output_by_index_llama.get(idx, ""))
df["mistral"] = df["index"].apply(lambda idx: output_by_index_mistral.get(idx, ""))
df.head()

,index,question,answer,category,macro category,gemma,llama,mistral
0,0,in una polmonite da staphylococcus aureus meti...,A,Area Preclinica 2016,farmacologia,A,A,A
1,1,i farmaci ipoglicemizzanti orali della classe ...,B,Area Preclinica 2016,farmacologia,B,B,B
2,2,"quale tra i seguenti farmaci antidiabetici, de...",D,Area Preclinica 2016,farmacologia,D,D,D
3,3,nello shock emorragico il trattamento iniziale...,A,Area Preclinica 2016,farmacologia,A,A,A
4,4,indicare quale dei seguenti farmaci antitumora...,A,Area Preclinica 2016,farmacologia,A,A,A


### Calcolo accuracy per ogni macro categoria

In [41]:
def compute_accuracy(df, prediction_column):
    return (df["answer"] == df[prediction_column]).mean()

# Accuracy per ciascuna macro categoria
accuracy_by_category = df.groupby("macro category").apply(
    lambda group: pd.Series({
        "accuracy_gemma": compute_accuracy(group, 'gemma'),
        "accuracy_llama": compute_accuracy(group, 'llama'),
        "accuracy_mistral": compute_accuracy(group, 'mistral'),
    })
).reset_index()

# Accuracy totale su tutto il dataset per ciascun modello
accuracy_total = pd.Series({
    "accuracy_gemma": compute_accuracy(df, 'gemma'),
    "accuracy_llama": compute_accuracy(df, 'llama'),
    "accuracy_mistral": compute_accuracy(df, 'mistral'),
}).to_frame(name="accuracy_total").T

# Mostra entrambi
print("Accuracy per categoria:")
display(accuracy_by_category)

print("Accuracy totale:")
display(accuracy_total)



Accuracy per categoria:


,macro category,accuracy_gemma,accuracy_llama,accuracy_mistral
0,chirurgia generale,0.777251,0.888626,0.810427
1,chirurgia specialistica,0.800885,0.889381,0.823009
2,farmacologia,0.832918,0.925187,0.882793
3,fisiologia,0.800681,0.893526,0.809199
4,medicina interna,0.847682,0.920530,0.837748
5,medicina legale e sanità pubblica,0.775100,0.859438,0.815261
6,medicina specialistica,0.823795,0.914910,0.845633
7,morfologia,0.754386,0.903509,0.795322
8,organi di senso,0.774390,0.914634,0.798780
9,ostetricia e ginecologia,0.758621,0.862069,0.750958


Accuracy totale:


,accuracy_gemma,accuracy_llama,accuracy_mistral
accuracy_total,0.808566,0.900932,0.828234


In [45]:
def compute_accuracy(df, prediction_column):
    return (df["answer"] == df[prediction_column]).mean()

# Accuracy per ciascuna macro categoria
accuracy_by_category = df.groupby("macro category").apply(
    lambda group: pd.Series({
        "accuracy_gemma": compute_accuracy(group, 'gemma'),
        "accuracy_llama": compute_accuracy(group, 'llama'),
        "accuracy_mistral": compute_accuracy(group, 'mistral'),
    })
).reset_index()

# Accuracy totale su tutto il dataset per ciascun modello
accuracy_total = pd.Series({
    "accuracy_gemma": compute_accuracy(df, 'gemma'),
    "accuracy_llama": compute_accuracy(df, 'llama'),
    "accuracy_mistral": compute_accuracy(df, 'mistral'),
}).to_frame(name="accuracy_total").T

# Lista delle colonne con accuracy numeriche
accuracy_columns = ["accuracy_gemma", "accuracy_llama", "accuracy_mistral"]

# Mostra la tabella per categoria con colorazione e formato
print("Accuracy per categoria:")
display(
    accuracy_by_category.style
        .background_gradient(subset=accuracy_columns, cmap='RdYlGn')
        .format({col: "{:.4f}" for col in accuracy_columns})  # solo su colonne numeriche
)

# Mostra la tabella totale
print("Accuracy totale:")
display(
    accuracy_total.style
        .background_gradient(cmap='RdYlGn', axis=None)
        .format({col: "{:.4f}" for col in accuracy_total.columns})
)


Accuracy per categoria:


,macro category,accuracy_gemma,accuracy_llama,accuracy_mistral
0,chirurgia generale,0.7773,0.8886,0.8104
1,chirurgia specialistica,0.8009,0.8894,0.8230
2,farmacologia,0.8329,0.9252,0.8828
3,fisiologia,0.8007,0.8935,0.8092
4,medicina interna,0.8477,0.9205,0.8377
5,medicina legale e sanità pubblica,0.7751,0.8594,0.8153
6,medicina specialistica,0.8238,0.9149,0.8456
7,morfologia,0.7544,0.9035,0.7953
8,organi di senso,0.7744,0.9146,0.7988
9,ostetricia e ginecologia,0.7586,0.8621,0.7510


Accuracy totale:


,accuracy_gemma,accuracy_llama,accuracy_mistral
accuracy_total,0.8086,0.9009,0.8282


In [47]:
# 1. Unione delle due tabelle per trovare min e max globali
combined = pd.concat([accuracy_preclinica, accuracy_clinica])
vmin = combined[accuracy_columns].min().min()
vmax = combined[accuracy_columns].max().max()

# 2. Visualizzazione con scala coerente
print("Accuracy per macro categoria (PRECLINICA):")
display(
    accuracy_preclinica.style
        .background_gradient(subset=accuracy_columns, cmap='RdYlGn', vmin=vmin, vmax=vmax)
        .format({col: "{:.4f}" for col in accuracy_columns})
)

print("Accuracy per macro categoria (CLINICA):")
display(
    accuracy_clinica.style
        .background_gradient(subset=accuracy_columns, cmap='RdYlGn', vmin=vmin, vmax=vmax)
        .format({col: "{:.4f}" for col in accuracy_columns})
)


Accuracy per macro categoria (PRECLINICA):


,macro category,accuracy_gemma,accuracy_llama,accuracy_mistral
0,farmacologia,0.8329,0.9252,0.8828
1,fisiologia,0.8007,0.8935,0.8092
2,morfologia,0.7544,0.9035,0.7953
3,patologia,0.8537,0.9177,0.8505
4,prevenzione e deontologia medica,0.8285,0.8882,0.8651


Accuracy per macro categoria (CLINICA):


,macro category,accuracy_gemma,accuracy_llama,accuracy_mistral
0,chirurgia generale,0.7773,0.8886,0.8104
1,chirurgia specialistica,0.8009,0.8894,0.8230
2,medicina interna,0.8477,0.9205,0.8377
3,medicina legale e sanità pubblica,0.7751,0.8594,0.8153
4,medicina specialistica,0.8238,0.9149,0.8456
5,organi di senso,0.7744,0.9146,0.7988
6,ostetricia e ginecologia,0.7586,0.8621,0.7510
7,pediatria,0.6842,0.8669,0.7430
8,radiologia e diagnostica,0.8783,0.9087,0.8739
